In [59]:
import pandas as pd
import requests
from edgar_functions import *
from bs4 import BeautifulSoup
from headers import headers

ticker = "WSM"

In [60]:
acc = get_filtered_filings(ticker, ten_k=True, just_accession_numbers=True)
acc_num = acc.iloc[0].replace("-", "")
acc_num


'000162828024012221'

In [66]:
get_statement_file_names_in_filing_summary(ticker, acc_num, headers=headers)


NameError: name 'get_statement_file_names_in_filing_summary' is not defined

In [67]:
def facts_DF(ticker, headers=headers):
    facts = get_facts(ticker, headers)
    us_gaap_data = facts["facts"]["us-gaap"]
    df_data = []
    for fact, details in us_gaap_data.items():
        for unit in details["units"]:
            for item in details["units"][unit]:
                row = item.copy()
                row["fact"] = fact
                df_data.append(row)

    df = pd.DataFrame(df_data)
    df["end"] = pd.to_datetime(df["end"])
    df["start"] = pd.to_datetime(df["start"])
    df = df.drop_duplicates(subset=["fact", "end", "val"])
    df.set_index("end", inplace=True)
    labels_dict = {fact: details["label"] for fact, details in us_gaap_data.items()}
    return df, labels_dict

In [68]:
facts, label_dict = facts_DF("WSM")

In [64]:
def annual_facts(ticker, headers=headers):
    accession_nums = edgar.get_filtered_filings(
        ticker, ten_k=True, just_accession_numbers=True
    )
    df, label_dict = facts_DF(ticker, headers)
    ten_k = df[df["accn"].isin(accession_nums)]
    ten_k = ten_k[ten_k.index.isin(accession_nums.index)]
    pivot = ten_k.pivot_table(values="val", columns="fact", index="end")
    pivot.rename(columns=label_dict, inplace=True)
    return pivot.T

In [65]:
def quarterly_facts(ticker, headers=headers):
    accession_nums = get_filtered_filings(
        ticker, ten_k=False, just_accession_numbers=True
    )
    df, label_dict = facts_DF(ticker, headers)
    ten_q = df[df["accn"].isin(accession_nums)]
    ten_q = ten_q[ten_q.index.isin(accession_nums.index)].reset_index(drop=False)
    ten_q = ten_q.drop_duplicates(subset=["fact", "end"], keep="last")
    pivot = ten_q.pivot_table(values="val", columns="fact", index="end")
    pivot.rename(columns=label_dict, inplace=True)
    return pivot.T

In [51]:
quarterly = quarterly_facts('WSM')


C:\Users\RIyer\AppData\Local\Temp\ipykernel_32708\4054876166.py:7: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  ten_q = ten_q[ten_q.index.isin(accession_nums.index)].reset_index(drop=False)


In [52]:
def save_dataframe_to_csv(dataframe, folder_name, ticker, statement_name, frequency):
    directory_path = os.path.join(folder_name, ticker)
    os.makedirs(directory_path, exist_ok=True)
    file_path = os.path.join(directory_path, f"{statement_name}_{frequency}.csv")
    dataframe.to_csv(file_path)
    return None